In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import os

In [30]:
auctions_data = pd.read_excel("raw_auctions_data.xlsx")

In [31]:
# The data set shape
print(f"The data set contains {auctions_data.shape[0]} records")
print(f"The data set contains {auctions_data.shape[1]} features")

The data set contains 1551 records
The data set contains 19 features


In [32]:
auctions_data["hasItemDesc"] = (auctions_data.conditionDescription.isna() == False).astype(int)

In [33]:
cols = ["condition", "ended_time_weekday", "ended_time_weekend", "price_dkk", "seller_country", "seller_feedback_pct", 
        "seller_feedback_positive", "seller_feedback_neutral", "seller_feedback_negative", "seller_member_duration", 
        "num_bidders", "bids", "duration", "hasItemDesc"]

Following columns are hereby removed

In [34]:
[item for item in list(auctions_data.columns) if item not in cols]

['title',
 'conditionDescription',
 'ended_time',
 'price',
 'seller_location',
 'seller_member_since']

In [35]:
nn_data = auctions_data.loc[:, cols]
nn_data

,condition,ended_time_weekday,ended_time_weekend,price_dkk,seller_country,seller_feedback_pct,seller_feedback_positive,seller_feedback_neutral,seller_feedback_negative,seller_member_duration,num_bidders,bids,duration,hasItemDesc
0,Used,2,0,5185,United States,100.0,6.0,0.0,0.0,1967,3,3,7,0
1,Used,2,0,5394,Germany,100.0,27.0,0.0,0.0,3894,2,3,5,0
2,Open box,2,0,6386,United Kingdom,100.0,29.0,0.0,0.0,5631,13,28,7,0
3,Used,2,0,6667,United States,100.0,8.0,0.0,0.0,332,18,44,7,0
4,Used,2,0,4512,United States,100.0,16.0,0.0,0.0,715,4,9,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546,Used,6,1,6066,Australia,100.0,5.0,0.0,0.0,5402,17,29,7,0
1547,Used,6,1,5886,Denmark,98.2,60.0,1.0,1.0,7272,21,52,7,1
1548,For parts or not working,6,1,3651,United States,100.0,26.0,0.0,0.0,3922,12,30,7,0
1549,Used,6,1,8791,United States,88.0,22.0,0.0,3.0,4033,43,101,10,1


**Remove countries with few observation**

In [36]:
nn_data = nn_data.groupby(["seller_country"]).filter(lambda x: x.condition.count() > 9)

In [37]:
# Checking
nn_data.groupby(["seller_country"]).count()

,condition,ended_time_weekday,ended_time_weekend,price_dkk,seller_feedback_pct,seller_feedback_positive,seller_feedback_neutral,seller_feedback_negative,seller_member_duration,num_bidders,bids,duration,hasItemDesc
seller_country,,,,,,,,,,,,,
Australia,106,106,106,106,100,102,102,102,106,106,106,106,106
Germany,222,222,222,222,205,219,219,219,222,222,222,222,222
Italy,37,37,37,37,29,32,32,32,37,37,37,37,37
United Kingdom,394,394,394,394,363,384,384,384,394,394,394,394,394
United States,743,743,743,743,642,704,704,704,743,743,743,743,743


**Remove obs with NA in feedback**

In [38]:
nn_data = nn_data[nn_data.seller_feedback_pct.isna() == False]
nn_data = nn_data[nn_data.seller_feedback_positive.isna() == False]

In [39]:
nn_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1336 entries, 0 to 1549
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   condition                 1336 non-null   object 
 1   ended_time_weekday        1336 non-null   int64  
 2   ended_time_weekend        1336 non-null   int64  
 3   price_dkk                 1336 non-null   int64  
 4   seller_country            1336 non-null   object 
 5   seller_feedback_pct       1336 non-null   float64
 6   seller_feedback_positive  1336 non-null   float64
 7   seller_feedback_neutral   1336 non-null   float64
 8   seller_feedback_negative  1336 non-null   float64
 9   seller_member_duration    1336 non-null   int64  
 10  num_bidders               1336 non-null   int64  
 11  bids                      1336 non-null   int64  
 12  duration                  1336 non-null   int64  
 13  hasItemDesc               1336 non-null   int32  
dtypes: float

In [41]:
nn_data.to_excel("data/COPY_ANN_data.xlsx", index = False)